In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install kaleido

     |████████████████████████████████| 79.9 MB 202 kB/s            


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import datasets, layers, models
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_curve, auc
import plotly.graph_objects as go
import glob
import os

In [4]:
def create_dir (direct):
  if not os.path.exists(direct):
    os.mkdir(direct)

In [5]:
ROOT_DIR = "/kaggle/input/ferset" #MT Directory
DATASET_02 = ROOT_DIR + "/02_FER"
DATASET_02_TRAIN = DATASET_02 + "/train"
DATASET_02_TEST = DATASET_02+ "/test"

In [6]:
import PIL

image = PIL.Image.open(DATASET_02_TRAIN + "/surprise/Training_99984132.jpg")
#image to open

width, height = image.size

print(image.mode)
print(width, height)

L
48 48


In [7]:
datagen = ImageDataGenerator()
image_size = (48, 48)
colormode = 'rgb'
test_set = datagen.flow_from_directory(DATASET_02_TEST, target_size=image_size, color_mode =colormode, batch_size=32, shuffle=False)
train_set = datagen.flow_from_directory(DATASET_02_TRAIN, target_size=image_size, color_mode =colormode, batch_size=32)

Found 7178 images belonging to 7 classes.
Found 28709 images belonging to 7 classes.


In [8]:
def create_model_VF():
    model = models.Sequential()
    model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    #model.add(layers.Dropout(0.4))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    #model.add(layers.Dropout(0.4))
    model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(7, activation='softmax'))
    
    return model



In [9]:
def create_model_VF_02():
    model = models.Sequential()
    model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.4))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.4))
    model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(7, activation='softmax'))
    
    return model



In [10]:
def create_model_VF_04():
    model = models.Sequential()
    model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.4))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.4))
    model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(7, activation='softmax'))
    
    return model


In [11]:
def get_data_df(dir):
  # modified code from: https://www.kaggle.com/namgalielei/simple-load-images-and-count-number-of-each-class

  train_df = pd.DataFrame()

  trainset = glob.glob(dir)
  #print(trainset)
  train_df['filename'] = [img.split("/")[-2]+"/"+img.split("/")[-1] for img in trainset]
  train_df['label'] = [img.split("/")[-2] for img in trainset]

  return train_df

In [12]:
test_train = get_data_df(DATASET_02_TRAIN+"/*/*.jpg")
test_train

,filename,label
0,surprise/Training_66056468.jpg,surprise
1,surprise/Training_26522394.jpg,surprise
2,surprise/Training_63239185.jpg,surprise
3,surprise/Training_67725241.jpg,surprise
4,surprise/Training_71308969.jpg,surprise
...,...,...
28704,happy/Training_18866880.jpg,happy
28705,happy/Training_16513509.jpg,happy
28706,happy/Training_10834907.jpg,happy
28707,happy/Training_30459679.jpg,happy


In [13]:
total = 0
value_list = []
for item in os.listdir(DATASET_02_TRAIN):
    y = len(os.listdir(DATASET_02_TRAIN+'/'+item))+len(os.listdir(DATASET_02_TEST+'/'+item))
    value_list.append(y)
    total += y
    print(item)
print(total)
print(value_list)
weights = {}
for n in range(0,len(value_list)):
    w = total/(len(value_list)*value_list[n])
    weights[n] = w
    
print(weights)

surprise
fear
angry
neutral
sad
disgust
happy
35887
[4002, 5121, 4953, 6198, 6077, 547, 8989]
{0: 1.2810380524023703, 1: 1.001115853488437, 2: 1.035072539009547, 3: 0.8271562255105334, 4: 0.8436258492207152, 5: 9.372420997649517, 6: 0.5703319930708962}


In [14]:
create_dir('/kaggle/working/Saved_CNN_Models')


In [15]:
from sklearn.model_selection import KFold, StratifiedKFold

train_data = get_data_df(DATASET_02_TRAIN+"/*/*.jpg")
Y = train_data[['label']]

#Y = final_train[['label']]
#n = len(final_train)   

skf = StratifiedKFold(n_splits = 5, random_state = 7, shuffle = True) 

#Un needed preprocessing aside from rescale
#idg = ImageDataGenerator(rescale=1./255)
create_dir('/kaggle/working/Saved_CNN_Models')
#model name
def get_model_name(k):
    return 'model_'+str(k)+'.h5'
def model_test(fold_count, epoch, model_method, image_size, mode):
    skf = StratifiedKFold(n_splits = fold_count, random_state = 7, shuffle = True)
    train_data = get_data_df(DATASET_02_TRAIN+"/*/*.jpg")
      #Incorperate preprocessing in the method 
      #####################################################################
      #aug_train = get_data_df(DATASET_01_ROOT+"/Aug_train/*/*.jpg")
      #aug_train['filename'] = aug_train['filename'].str[7:]
      #final_train = test_train.append(aug_train, sort=False, ignore_index=True)


      #######################################################################
      #idg = ImageDataGenerator(rescale=1./255)
    idg = ImageDataGenerator()
    Y = train_data[['label']]
    n = len(train_data)    
    VALIDATION_ACCURACY = []
    VALIDATION_LOSS = []

      #create a directory
    save_dir = '/kaggle/working/Saved_CNN_Models/'+model_method.__name__+'_'+str(epoch)+'epochs/'
    create_dir(save_dir)
    
    img_dir = DATASET_02_TRAIN

    fold_var = 1

      #n is number of samples, doesn't care about x values for split, just y
    for train_index, val_index in skf.split(np.zeros(n),Y):

        #Index value for fold
        training_data = train_data.iloc[train_index]
        validation_data = train_data.iloc[val_index]
        #generator for train
        print("hi")
        print(validation_data.head())
        #Modify the training size, have an input adjustability 
        train_data_generator = idg.flow_from_dataframe(training_data, directory = img_dir,
                                                       target_size=image_size, batch_size=32, color_mode =mode,
                                                       x_col = "filename", y_col = "label",shuffle = True)
        #print("hi")
        #generator for validation
        valid_data_generator  = idg.flow_from_dataframe(validation_data, directory = img_dir,
                                                        target_size=image_size, color_mode =mode,batch_size=32,
                                                        x_col = "filename", y_col = "label",shuffle = True)
        #print("hi")
        #create a method for this
        # CREATE NEW MODEL
        model = model_method()
        # COMPILE NEW MODEL
        model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                optimizer= tf.keras.optimizers.Adam(learning_rate=0.0001),
                metrics=['accuracy'])

        # CREATE CALLBACKS
        #Unsure about checpoint
        checkpoint = tf.keras.callbacks.ModelCheckpoint(save_dir+model_method.__name__+"_"+get_model_name(fold_var), 
                    monitor='val_accuracy', verbose=1, 
                    save_best_only=True, mode='max')

        callbacks_list = [checkpoint]
        # There can be other callbacks, but just showing one because it involves the model name
        # This saves the best model
        # FIT THE MODEL
        #Include the weights for classes
        history = model.fit(train_data_generator,
                epochs=epoch,
                callbacks=callbacks_list,
                validation_data=valid_data_generator
                ,verbose = 2)

        #PLOT HISTORY
        #Plots for whichever
        #		:
        #plt.plot(history.history['binary_accuracy'], label='accuracy')
        #plt.plot(history.history['val_binary_accuracy'], label = 'val_accuracy')
        #plt.xlabel('Epoch')
        #plt.ylabel('Accuracy')
        #plt.ylim([0.3, 1])
        #plt.legend(loc='lower right')
        #plt.show()
        #frame = {'Accuracy': history.history['binary_accuracy'], 'Val_Accuracy': history.history['val_binary_accuracy']}
        #fig = px.line(frame)
        #fig.show()
        fig = go.Figure()
        fig.add_trace(go.Scatter(y= history.history['accuracy'],
                          mode='lines+markers',
                          name='Training Accuracy'))
        fig.add_trace(go.Scatter(y= history.history['val_accuracy'],
                          mode='lines+markers',
                          name='Validation Accuracy'))
        fig.update_yaxes(range=[0, 1])
        fig.update_layout(
            title="Fold "+ str(fold_var)+ "'s performance",
            title_x=0.5,
            xaxis_title="Epochs",
            yaxis_title="Accuracy",
            autosize=False,
            width=800,
            height=400,
            margin=dict(l=20, r=20, t=40, b=20),)
        fig.show()
        graph_dir = '/kaggle/working/img'
        create_dir(graph_dir)
        fig.write_image(graph_dir+'/fold'+str(fold_var)+'.png') 

        #		:
        # LOAD BEST MODEL to evaluate the performance of the model
        model.load_weights(save_dir+model_method.__name__+"_" + "model_"+str(fold_var)+".h5")

        results = model.evaluate(test_set)
        results = dict(zip(model.metrics_names,results))

        VALIDATION_ACCURACY.append(results['accuracy'])
        VALIDATION_LOSS.append(results['loss'])

        tf.keras.backend.clear_session()

        fold_var += 1

In [16]:
#model_test(5, 1, create_model_V1, (48,48), 'rgb')

In [17]:
#model_test(5, 30, create_model_VF, (48,48,), 'rgb')

In [18]:
model_test(5, 30, create_model_VF_02, (48,48,), 'rgb')

hi
                          filename     label
10  surprise/Training_52655852.jpg  surprise
16  surprise/Training_44186778.jpg  surprise
21  surprise/Training_87248788.jpg  surprise
31  surprise/Training_32069005.jpg  surprise
45   surprise/Training_4473463.jpg  surprise
Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.


2022-05-06 03:23:55.924660: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-05-06 03:23:56.376834: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/30
718/718 - 206s - loss: 6.1189 - accuracy: 0.1841 - val_loss: 1.9285 - val_accuracy: 0.1759

Epoch 00001: val_accuracy improved from -inf to 0.17590, saving model to /kaggle/working/Saved_CNN_Models/create_model_VF_02_30epochs/create_model_VF_02_model_1.h5
Epoch 2/30
718/718 - 105s - loss: 1.8774 - accuracy: 0.2127 - val_loss: 1.8889 - val_accuracy: 0.2264

Epoch 00002: val_accuracy improved from 0.17590 to 0.22640, saving model to /kaggle/working/Saved_CNN_Models/create_model_VF_02_30epochs/create_model_VF_02_model_1.h5
Epoch 3/30
718/718 - 103s - loss: 1.8276 - accuracy: 0.2504 - val_loss: 1.8941 - val_accuracy: 0.2240

Epoch 00003: val_accuracy did not improve from 0.22640
Epoch 4/30
718/718 - 103s - loss: 1.8194 - accuracy: 0.2513 - val_loss: 1.8928 - val_accuracy: 0.2290

Epoch 00004: val_accuracy improved from 0.22640 to 0.22901, saving model to /kaggle/working/Saved_CNN_Models/create_model_VF_02_30epochs/create_model_VF_02_model_1.h5
Epoch 5/30
718/718 - 102s - loss: 1

225/225 [==============================] - 50s 222ms/step - loss: 1.2191 - accuracy: 0.5315
hi
                          filename     label
2   surprise/Training_63239185.jpg  surprise
6   surprise/Training_69351834.jpg  surprise
7   surprise/Training_67440750.jpg  surprise
12  surprise/Training_46386777.jpg  surprise
18  surprise/Training_43664635.jpg  surprise
Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.
Epoch 1/30
718/718 - 107s - loss: 4.6689 - accuracy: 0.1840 - val_loss: 1.9044 - val_accuracy: 0.1930

Epoch 00001: val_accuracy improved from -inf to 0.19296, saving model to /kaggle/working/Saved_CNN_Models/create_model_VF_02_30epochs/create_model_VF_02_model_2.h5
Epoch 2/30
718/718 - 106s - loss: 1.8510 - accuracy: 0.2289 - val_loss: 1.8683 - val_accuracy: 0.2550

Epoch 00002: val_accuracy improved from 0.19296 to 0.25496, saving model to /kaggle/working/Saved_CNN_Models/create_model_VF_02_30epochs/creat

225/225 [==============================] - 18s 79ms/step - loss: 1.2132 - accuracy: 0.5684
hi
                          filename     label
4   surprise/Training_71308969.jpg  surprise
8   surprise/Training_93942694.jpg  surprise
9   surprise/Training_11693705.jpg  surprise
22  surprise/Training_11550894.jpg  surprise
23  surprise/Training_91279496.jpg  surprise
Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.
Epoch 1/30
718/718 - 108s - loss: 4.1476 - accuracy: 0.1983 - val_loss: 1.8621 - val_accuracy: 0.2687

Epoch 00001: val_accuracy improved from -inf to 0.26872, saving model to /kaggle/working/Saved_CNN_Models/create_model_VF_02_30epochs/create_model_VF_02_model_3.h5
Epoch 2/30
718/718 - 108s - loss: 1.8224 - accuracy: 0.2586 - val_loss: 1.7763 - val_accuracy: 0.3299

Epoch 00002: val_accuracy improved from 0.26872 to 0.32985, saving model to /kaggle/working/Saved_CNN_Models/create_model_VF_02_30epochs/create

225/225 [==============================] - 18s 80ms/step - loss: 1.1900 - accuracy: 0.5780
hi
                          filename     label
0   surprise/Training_66056468.jpg  surprise
3   surprise/Training_67725241.jpg  surprise
11  surprise/Training_62829838.jpg  surprise
14  surprise/Training_97275390.jpg  surprise
19  surprise/Training_29912845.jpg  surprise
Found 22967 validated image filenames belonging to 7 classes.
Found 5742 validated image filenames belonging to 7 classes.
Epoch 1/30
718/718 - 107s - loss: 4.5248 - accuracy: 0.2000 - val_loss: 1.8873 - val_accuracy: 0.2468

Epoch 00001: val_accuracy improved from -inf to 0.24678, saving model to /kaggle/working/Saved_CNN_Models/create_model_VF_02_30epochs/create_model_VF_02_model_4.h5
Epoch 2/30
718/718 - 108s - loss: 1.8441 - accuracy: 0.2510 - val_loss: 1.8446 - val_accuracy: 0.2807

Epoch 00002: val_accuracy improved from 0.24678 to 0.28074, saving model to /kaggle/working/Saved_CNN_Models/create_model_VF_02_30epochs/create

225/225 [==============================] - 16s 70ms/step - loss: 1.2373 - accuracy: 0.5807
hi
                          filename     label
1   surprise/Training_26522394.jpg  surprise
5   surprise/Training_49242724.jpg  surprise
13   surprise/Training_7022565.jpg  surprise
15  surprise/Training_37026288.jpg  surprise
17  surprise/Training_97667504.jpg  surprise
Found 22968 validated image filenames belonging to 7 classes.
Found 5741 validated image filenames belonging to 7 classes.
Epoch 1/30
718/718 - 108s - loss: 6.1230 - accuracy: 0.1910 - val_loss: 1.9140 - val_accuracy: 0.1930

Epoch 00001: val_accuracy improved from -inf to 0.19300, saving model to /kaggle/working/Saved_CNN_Models/create_model_VF_02_30epochs/create_model_VF_02_model_5.h5
Epoch 2/30
718/718 - 108s - loss: 1.8943 - accuracy: 0.2216 - val_loss: 1.8936 - val_accuracy: 0.2538

Epoch 00002: val_accuracy improved from 0.19300 to 0.25379, saving model to /kaggle/working/Saved_CNN_Models/create_model_VF_02_30epochs/create

225/225 [==============================] - 16s 71ms/step - loss: 1.1622 - accuracy: 0.5762


In [19]:
#model_test(5, 30, create_model_VF_04, (48,48,), 'rgb')